In [15]:
!pip install bs4 
!pip install lxml
!pip install pandas
!pip install random


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree
import time
from random import randint

In [17]:
response = requests.get("https://wslwebservices.leg.wa.gov/LegislationService.asmx/GetLegislationByYear?year=2023")

xml_data = response.content

root = etree.fromstring(xml_data)

In [18]:
root[0][0].text

'2023-24'

In [19]:
# Define the XML namespace
ns = {'ns': 'http://WSLWebServices.leg.wa.gov/'}

# Extract information
data = []
for info in root.findall('ns:LegislationInfo', ns):
    biennium = info.find('ns:Biennium', ns)
    bill_id = info.find('ns:BillId', ns)
    bill_number = info.find('ns:BillNumber', ns)
    bill_type = info.find('.//ns:ShortLegislationType/ns:LongLegislationType', ns)
    agency = info.find('ns:OriginalAgency', ns)

    # Prepare the data dictionary safely
    data.append({
        'Biennium': biennium.text if biennium is not None else None,
        'Bill ID': bill_id.text if bill_id is not None else None,
        'Bill Number': bill_number.text if bill_number is not None else None,
        'Bill Type': bill_type.text if bill_type is not None else None,
        'Agency': agency.text if agency is not None else None
    })

# Create a DataFrame
df = pd.DataFrame(data)
print(df)


     Biennium   Bill ID Bill Number                  Bill Type  Agency
0     2023-24   HB 1000        1000                       Bill   House
1     2023-24   HB 1001        1001                       Bill   House
2     2023-24   HB 1002        1002                       Bill   House
3     2023-24   HB 1003        1003                       Bill   House
4     2023-24  SHB 1003        1003                       Bill   House
...       ...       ...         ...                        ...     ...
3289  2023-24  SGA 9353        9353  Gubernatorial Appointment  Senate
3290  2023-24  SGA 9354        9354  Gubernatorial Appointment  Senate
3291  2023-24  SGA 9355        9355  Gubernatorial Appointment  Senate
3292  2023-24  SGA 9356        9356  Gubernatorial Appointment  Senate
3293  2023-24  SGA 9357        9357  Gubernatorial Appointment  Senate

[3294 rows x 5 columns]


In [20]:
df[['Bill Type', 'Agency']].groupby(by=['Bill Type', 'Agency']).sum()

Empty DataFrame
Columns: []
Index: [(Bill, House), (Bill, Senate), (Concurrent Resolution, House), (Concurrent Resolution, Senate), (Gubernatorial Appointment, Senate), (Joint Memorial, House), (Joint Memorial, Senate), (Joint Resolution, House), (Joint Resolution, Senate), (Resolution, House), (Resolution, Senate)]

In [21]:
# drop the duplicates to keep things simple... for now
subset_df = df.drop_duplicates(subset=['Bill Number'])

In [22]:
senate_df = subset_df.loc[df['Agency'] == 'Senate']
house_df = subset_df.loc[df['Agency'] == 'House']

In [23]:
def get_bill_text(bill_number, type, agency):
  if agency == 'House':
    url = f'https://lawfilesext.leg.wa.gov/biennium/2023-24/Htm/Bills/House%20Bills/{bill_number}.htm'
  elif agency == 'Senate':
    url = f'https://lawfilesext.leg.wa.gov/biennium/2023-24/Htm/Bills/Senate%20Bills/{bill_number}.htm'

  # Generate a random number of seconds to wait
  random_seconds = randint(1, 3)  # Random wait between 1 and 3 seconds

  # Wait for the random number of seconds
  time.sleep(random_seconds)

  # Send HTTP request to the URL
  response = requests.get(url)

  # Ensure the request was successful
  if response.status_code == 200:
      # Parse the HTML content of the page using BeautifulSoup
      soup = BeautifulSoup(response.text, 'html.parser')

      # Extract and print the text from the parsed HTML
      text = soup.get_text(separator=' ', strip=True)
      print(f'Appended {bill_number} text to df')
      return text
  else:
      return None


In [24]:
get_bill_text(1000, 'Bill', 'House')

Appended 1000 text to df


'ï»¿ H-0113.1 HOUSE BILL 1000 State of Washington 68th Legislature 2023 Regular Session By Representatives Stokesbary, Corry, Barkis, Graham, Griffey, Robertson, and Caldier Prefiled 12/05/22. Read first time 01/09/23. Referred to Committee on Finance. AN ACT Relating to providing sales tax relief by expanding the working families\' tax credit; reenacting and amending RCW 82.08.0206 ; and providing an effective date. BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF WASHINGTON: Sec. 1. RCW 82.08.0206 and 2022 c 41 s 1 and 2022 c 33 s 1 are each reenacted and amended to read as follows: (1) A working families\' tax credit, in the form of a refund of tax due under this chapter and chapter 82.12 RCW, is provided to eligible low-income persons for sales and use taxes paid under this chapter and chapter 82.12 RCW after January 1, 2022. (2) For purposes of the credit in this section, the following definitions apply: (a)(i) "Eligible low-income person" means an individual who: (A) Is eligible 

In [25]:
house_df['Bill Text'] = subset_df.apply(lambda x: get_bill_text(x['Bill Number'], x['Bill Type'], x['Agency']), axis=1)

Appended 1000 text to df
Appended 1001 text to df
Appended 1002 text to df
Appended 1003 text to df
Appended 1004 text to df
Appended 1005 text to df
Appended 1006 text to df
Appended 1007 text to df
Appended 1008 text to df
Appended 1009 text to df
Appended 1010 text to df
Appended 1011 text to df
Appended 1012 text to df
Appended 1013 text to df
Appended 1014 text to df
Appended 1015 text to df
Appended 1016 text to df
Appended 1017 text to df
Appended 1018 text to df
Appended 1019 text to df
Appended 1020 text to df
Appended 1021 text to df
Appended 1022 text to df
Appended 1023 text to df
Appended 1024 text to df
Appended 1025 text to df
Appended 1026 text to df
Appended 1027 text to df
Appended 1028 text to df
Appended 1029 text to df
Appended 1030 text to df
Appended 1031 text to df
Appended 1032 text to df
Appended 1033 text to df
Appended 1034 text to df
Appended 1035 text to df
Appended 1036 text to df
Appended 1037 text to df
Appended 1038 text to df
Appended 1039 text to df


KeyboardInterrupt: 

In [ ]:
house_df

,Biennium,Bill ID,Bill Number,Bill Type,Agency,Bill Text
0,2023-24,HB 1000,1000,Bill,House,ï»¿ H-0113.1 HOUSE BILL 1000 State of Washingt...
1,2023-24,HB 1001,1001,Bill,House,ï»¿ H-0082.1 HOUSE BILL 1001 State of Washingt...
2,2023-24,HB 1002,1002,Bill,House,ï»¿ H-0034.1 HOUSE BILL 1002 State of Washingt...
3,2023-24,HB 1003,1003,Bill,House,ï»¿ H-0142.1 HOUSE BILL 1003 State of Washingt...
5,2023-24,HB 1004,1004,Bill,House,ï»¿ H-0031.1 HOUSE BILL 1004 State of Washingt...
...,...,...,...,...,...,...
1570,2023-24,HR 4647,4647,Resolution,House,None
1571,2023-24,HR 4648,4648,Resolution,House,None
1572,2023-24,HR 4649,4649,Resolution,House,None
1573,2023-24,HR 4650,4650,Resolution,House,None


In [ ]:
house_df.to_csv('house_df.csv')

In [ ]:
senate_df = senate_df.loc[senate_df['Bill Type'] == 'Bill']

In [ ]:
senate_df['Bill Text'] = subset_df.apply(lambda x: get_bill_text(x['Bill Number'], x['Bill Type'], x['Agency']), axis=1)

KeyboardInterrupt: 

In [ ]:
senate_df

,Biennium,Bill ID,Bill Number,Bill Type,Agency
1575,2023-24,SB 5000,5000,Bill,Senate
1576,2023-24,SB 5001,5001,Bill,Senate
1579,2023-24,SB 5002,5002,Bill,Senate
1582,2023-24,SB 5003,5003,Bill,Senate
1583,2023-24,SB 5004,5004,Bill,Senate
...,...,...,...,...,...
2853,2023-24,SB 5925,5925,Bill,Senate
2854,2023-24,SB 5926,5926,Bill,Senate
2855,2023-24,SB 5927,5927,Bill,Senate
2856,2023-24,SB 5928,5928,Bill,Senate


In [ ]:
subset_df = pd.concat(house_df, senate_df)

In [ ]:
subset_df['Summary'] = None

In [ ]:
subset_df

In [ ]:
subset_df.to_csv('bills.csv')

In [ ]:
hb = pd.read_csv('house_df.csv')

In [ ]:
hb['Summary'] = None

In [26]:
hb

,Unnamed: 0.1,Unnamed: 0,Biennium,Bill ID,Bill Number,Bill Type,Agency,Bill Text,Summary
0,0,0,2023-24,HB 1000,1000,Bill,House,ï»¿ H-0113.1 HOUSE BILL 1000 State of Washingt...,NaN
1,1,1,2023-24,HB 1001,1001,Bill,House,ï»¿ H-0082.1 HOUSE BILL 1001 State of Washingt...,NaN
2,2,2,2023-24,HB 1002,1002,Bill,House,ï»¿ H-0034.1 HOUSE BILL 1002 State of Washingt...,NaN
3,3,3,2023-24,HB 1003,1003,Bill,House,ï»¿ H-0142.1 HOUSE BILL 1003 State of Washingt...,NaN
4,4,5,2023-24,HB 1004,1004,Bill,House,ï»¿ H-0031.1 HOUSE BILL 1004 State of Washingt...,NaN
...,...,...,...,...,...,...,...,...,...
1063,1063,1497,2023-24,HB 2063,2063,Bill,House,ï»¿ H-2323.1 HOUSE BILL 2063 State of Washingt...,NaN
1064,1064,1498,2023-24,HB 2064,2064,Bill,House,ï»¿ H-2325.1 HOUSE BILL 2064 State of Washingt...,NaN
1065,1065,1499,2023-24,HB 2065,2065,Bill,House,ï»¿ H-2336.1 HOUSE BILL 2065 State of Washingt...,NaN
1066,1066,1500,2023-24,HB 2066,2066,Bill,House,ï»¿ H-2377.1 HOUSE BILL 2066 State of Washingt...,NaN


In [29]:
hb.to_csv('house_df.csv')

In [12]:
hb = hb.loc[hb['Bill Type'] == 'Bill']

In [28]:
hb = hb[['Biennium', 'Bill ID', 'Bill Number',
       'Bill Type', 'Agency', 'Bill Text', 'Summary']]